# Эксперимент: генерация TVR из тестовой конфигурации

### Шаг 1. Определяем список колонок и подключаем зависимости
Определяем перечень override-колонок, указанный в задаче, и настраиваем окружение для работы с проектом.

In [ ]:
from pathlib import Path
import sys

import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path.cwd().resolve()
SRC_PATH = PROJECT_ROOT / 'src'
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

override_columns = [
    'base_long_SM',
    'base_long_Sec_0',
    'base_long_Sec_1',
    'base_long_V_0',
    'base_short_SM',
    'base_short_Sec_0',
    'base_short_Sec_1',
    'base_short_V_0',
    'filter_1_long_short_Sec_0',
    'filter_1_long_short_Sec_1',
    'filter_2_long_short_Sec_0',
    'filter_2_long_short_W_1',
    'filter_3_long_short_2_Sec_0',
    'filter_3_long_short_2_Sec_1',
    'filter_3_long_short_Sec_0',
    'filter_3_long_short_W_1',
    'filter_4_long_short_Sec_0',
    'filter_4_long_short_Sec_1',
    'filter_5_long_short_Sec_0',
    'filter_5_long_short_Sec_1',
    'filter_6_long_short_Sec_0',
    'filter_6_long_short_W_1',
    'filter_7_long_short_Sec_0',
    'filter_7_long_short_W_1',
]

display(pd.Series(override_columns, name='override_columns'))

### Шаг 2. Формируем тестовую конфигурационную таблицу
Создаём DataFrame с указанными колонками и добавляем служебные поля strategy_id, start, sec_0.

In [ ]:
num_rows = 3
base_sequence = range(1, num_rows + 1)

data = {column: [value + idx for value in base_sequence] for idx, column in enumerate(override_columns)}

config_df = pd.DataFrame(data)
config_df['strategy_id'] = [f'S{idx + 1}' for idx in range(num_rows)]
config_df['start'] = [100 + idx * 50 for idx in range(num_rows)]
config_df['sec_0'] = [1, 1, 2]

ordered_columns = ['strategy_id', 'start', 'sec_0'] + override_columns
config_df = config_df[ordered_columns]

display(config_df)

### Шаг 3. Строим шаблон и генерируем экспериментальный TVR
Загружаем маску, конструируем StrategyTemplate, запускаем StrategyGenerator и смотрим результат.

In [ ]:
from tvr_service.templates.mask_template import build_template_from_mask_df
from tvr_service.generator.strategy_builder import StrategyGenerator

mask_path = Path('docs/mask_template.xlsx')
raw_mask_df = pd.read_excel(mask_path)
mask_df = raw_mask_df.dropna(subset=['row_alias']).copy()

mask_result = build_template_from_mask_df(
    mask_df,
    name='mask_template',
    row_alias_column='row_alias',
    marker=1,
)

generator = StrategyGenerator(mask_result.template)
tvr_df = generator.generate(config_df)

print('????? override-??????? ? ???????:', len(mask_result.override_columns))
display(pd.Series(mask_result.override_columns[:15], name='override_sample'))

display(tvr_df.head(40))